In [1]:
import yfinance as yf
import pandas as pd
from tilib import indicators as id
import numpy as np

In [2]:
btc_df = yf.Ticker('^SPX')
btc_df.info
hist = btc_df.history(start = '2023-11-25', end = '2024-01-18' , interval = '5m') #period = '1w'
df = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']]).round(2)
df_cumulative_delta = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']]).round(2) 
df.head()

,Volume,Close,High,Low
Datetime,,,,
2023-11-27 09:30:00-05:00,0,4549.27,4558.49,4549.23
2023-11-27 09:35:00-05:00,30782673,4549.93,4550.29,4546.32
2023-11-27 09:40:00-05:00,29432485,4548.67,4550.23,4547.90
2023-11-27 09:45:00-05:00,28696450,4548.38,4550.11,4547.78
2023-11-27 09:50:00-05:00,25111720,4549.70,4549.75,4546.46


In [3]:
#Calcualting HLC3 value
df['HLC3'] = (df['High'] + df['Low'] + df['Close']) / 3

# Calculating VWAP (using HLC3)
df['VWAP'] = (df['HLC3'] * df['Volume']).cumsum() / df['Volume'].cumsum()

#Calculating Standard Deviation 
df['PriceDeviation'] = (df['HLC3'] - df['VWAP']).rolling(window = 78).std() ##Window 78 set based on 5 min trading data (78 5 min in 1 day)##
# Session based (How it was on Pinescript) appears to be based on daily 

# Calculate bands [UpperBand_1,	LowerBand_1, UpperBand_2, LowerBand_2, UpperBand_3, LowerBand_3]
for i in [1, 2, 3]:
    df[f'UpperBand_{i}'] = df['VWAP'] + (i * df['PriceDeviation'])
    df[f'LowerBand_{i}'] = df['VWAP'] - (i * df['PriceDeviation'])

df = df.dropna()
df_hlc3_vwap = df
df_hlc3_vwap

,Volume,Close,High,Low,HLC3,VWAP,PriceDeviation,UpperBand_1,LowerBand_1,UpperBand_2,LowerBand_2,UpperBand_3,LowerBand_3
Datetime,,,,,,,,,,,,,
2023-11-28 09:30:00-05:00,57510711,4544.34,4547.30,4544.01,4545.216667,4553.425398,2.726457,4556.151855,4550.698941,4558.878312,4547.972484,4561.604769,4545.246027
2023-11-28 09:35:00-05:00,29678406,4544.82,4546.51,4542.47,4544.600000,4553.258020,2.957044,4556.215064,4550.300976,4559.172108,4547.343932,4562.129152,4544.386888
2023-11-28 09:40:00-05:00,31675830,4541.39,4544.72,4540.73,4542.280000,4553.040214,3.260875,4556.301089,4549.779339,4559.561964,4546.518463,4562.822840,4543.257588
2023-11-28 09:45:00-05:00,36061799,4543.98,4545.37,4541.26,4543.536667,4552.830295,3.471699,4556.301994,4549.358596,4559.773693,4545.886897,4563.245391,4542.415198
2023-11-28 09:50:00-05:00,26478742,4540.98,4544.37,4540.85,4542.066667,4552.658509,3.716534,4556.375044,4548.941975,4560.091578,4545.225440,4563.808112,4541.508906
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-17 15:35:00-05:00,30275814,4732.30,4733.30,4728.43,4731.343333,4682.552841,9.187009,4691.739850,4673.365832,4700.926859,4664.178822,4710.113869,4654.991813
2024-01-17 15:40:00-05:00,32826389,4732.05,4733.92,4731.63,4732.533333,4682.577303,8.717203,4691.294506,4673.860101,4700.011709,4665.142898,4708.728911,4656.425696
2024-01-17 15:45:00-05:00,42493578,4733.41,4734.87,4732.03,4733.436667,4682.609506,8.179444,4690.788950,4674.430063,4698.968393,4666.250619,4707.147837,4658.071176


In [ ]:
df_cumulative_delta

In [ ]:
# # Calculate Cumulative Delta
# data['BuyVolume'] = np.where(data['Close'] >= data['Open'], data['Volume'], 0)
# data['SellVolume'] = np.where(data['Close'] < data['Open'], data['Volume'], 0)
# data['Delta'] = data['BuyVolume'] - data['SellVolume']
# data['CumulativeDelta'] = data['Delta'].cumsum()

# # Calculate Moving Averages
# maLength1 = 20
# maLength2 = 50
# data['MA_CumDelta_1'] = data['CumulativeDelta'].rolling(window=maLength1).mean()
# data['MA_CumDelta_2'] = data['CumulativeDelta'].rolling(window=maLength2).mean()

# # VWAP calculation
# # Note: VWAP typically resets daily (or on your chosen timeframe)
# data['VWAP'] = (data['Volume'] * (data['High'] + data['Low'] + data['Close']) / 3).cumsum() / data['Volume'].cumsum()

# # Detect Divergences
# # Simplified example - you might need more sophisticated logic
# data['BullishDivergence'] = (data['Close'] > data['Open']) & (data['Delta'] < data['Delta'].shift(1))
# data['BearishDivergence'] = (data['Close'] < data['Open']) & (data['Delta'] > data['Delta'].shift(1))